In [1]:
import os
from typing import Optional
from sqlmodel import Field, SQLModel, create_engine, Session
import pathlib
from IPython.display import display, Markdown
from kowinski.parser.repo_parser import parse_repository
from kowinski.parser.file_parser import analyze_python_files
from kowinski.agents.code_agent import create_analysis_agent, create_code_agent, create_model, create_patch_agent
import pandas as pd 
import sqlite3
from smolagents import load_tool, CodeAgent, HfApiModel, DuckDuckGoSearchTool, OpenAIServerModel, LiteLLMModel, tool
from vllm import LLM
from dotenv import load_dotenv
import base64
load_dotenv()

LANGFUSE_PUBLIC_KEY=os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY=os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"


True

In [2]:
import difflib

In [3]:
issue_data = pd.read_parquet('../data/data.parquet')
row = issue_data.iloc[0]
repo_path = f'../data/repos/repo__{row["instance_id"]}'

In [4]:
model = create_model(
)

# Create patch agent
patch_agent = create_patch_agent(
    model=model,
    db_path='repository.db'
)

task = f"""You are tasked with understanding an issue in the following github repository {row['repo']}.
You will be given a description of the issue and tools to search the codebase.
Your job is to determine, what file or files are causing the issue.
The tools provided will allow you to search the codebase and find the files that are causing the issue.
Determine what is the function or class where this issue is happening
### Issue Description
{row['problem_statement']}"""
# Run the agent

In [5]:
Markdown(patch_agent.system_prompt)

You are an expert software engineer assistant who specializes in analyzing and fixing code issues. Your task is to generate patches to fix GitHub issues in code repositories.

To do so, you have been given access to a set of tools to help you understand code files, analyze issues, and create patches.

To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards understanding the issue and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using the patch generation tools:

---
Task: "Fix this issue in the file utils/auth.py:

Bug report: Authentication fails when username contains an '@' symbol. The app crashes with the error:

```
Traceback (most recent call last):
  File 'app.py', line 67, in login
    user = authenticate(username, password)
  File 'utils/auth.py', line 42, in authenticate
    parts = username.split('@')
    email = '@'.join(parts[1:])
IndexError: list index out of range
```

The issue is that the code assumes all usernames have an '@' symbol, but they don't."

Thought: I need to investigate the authenticate function in utils/auth.py. The traceback shows that the error happens on line 42, where the code tries to access parts[1:], but parts might be empty if the username doesn't contain an '@' symbol.
Code:
```py
# First, let's look at the file content to understand the context
file_content = get_file_content("utils/auth.py")
print(file_content)
```<end_code>
Observation:
```python
# utils/auth.py
import hashlib
from database import get_user_by_email

def authenticate(username, password):
    """
    Authenticate a user.
    
    Args:
        username: The username to authenticate
        password: The user's password
        
    Returns:
        User object if authentication successful, None otherwise
    """
    # Extract email from username (assuming username is email)
    parts = username.split('@')
    email = '@'.join(parts[1:])
    
    if not email:
        return None
    
    # Get user from database
    user = get_user_by_email(email)
    if not user:
        return None
    
    # Validate password
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    if user.password != hashed_password:
        return None
    
    return user
```

Thought: Now I see the issue. In the authenticate function, it splits the username by '@' and assumes there's at least one '@' in the username. The issue happens when a username doesn't contain '@'. Let me use the extract_method_context tool to get more context around this function.
Code:
```py
method_context = extract_method_context(file_content, "authenticate")
print(method_context)
```<end_code>
Observation:
{
  "found": true,
  "start_line": 4,
  "end_line": 31,
  "content": "def authenticate(username, password):\n    \"\"\"\n    Authenticate a user.\n    \n    Args:\n        username: The username to authenticate\n        password: The user's password\n        \n    Returns:\n        User object if authentication successful, None otherwise\n    \"\"\"\n    # Extract email from username (assuming username is email)\n    parts = username.split('@')\n    email = '@'.join(parts[1:])\n    \n    if not email:\n        return None\n    \n    # Get user from database\n    user = get_user_by_email(email)\n    if not user:\n        return None\n    \n    # Validate password\n    hashed_password = hashlib.sha256(password.encode()).hexdigest()\n    if user.password != hashed_password:\n        return None\n    \n    return user",
  "context_before": "# utils/auth.py\nimport hashlib\nfrom database import get_user_by_email",
  "context_after": "",
  "full_range": [3, 30]
}

Thought: Now let's identify patterns in the file that might be related to the issue.
Code:
```py
issue_patterns = identify_issue_patterns(file_content, "Bug report: Authentication fails when username contains an '@' symbol. The app crashes with the error:\n\nTraceback (most recent call last):\n  File 'app.py', line 67, in login\n    user = authenticate(username, password)\n  File 'utils/auth.py', line 42, in authenticate\n    parts = username.split('@')\n    email = '@'.join(parts[1:])\nIndexError: list index out of range")
print(issue_patterns)
```<end_code>
Observation:
[
  {
    "type": "function_definition",
    "name": "authenticate",
    "line": 4,
    "content": "def authenticate(username, password):"
  },
  {
    "type": "traceback_line",
    "line": 42,
    "content": "    email = '@'.join(parts[1:])",
    "context": "  File 'utils/auth.py', line 42, in authenticate\n    parts = username.split('@')\n    email = '@'.join(parts[1:])\n"
  }
]

Thought: I've identified the issue. When a username doesn't contain an '@' symbol, the parts list will just have one element, and parts[1:] will be an empty list. When trying to join an empty list with '@', there's no issue, but the resulting email variable will be empty.

The fix should check if the username contains an '@' symbol before trying to extract the email part. If it doesn't, we should handle that case appropriately. Let me create a patch operation to fix this.
Code:
```py
# Define the patch operations
patch_operations = [
  {
    "operation": "replace",
    "start_line": 13,
    "end_line": 15,
    "content": "    # Extract email from username (assuming username is email)\n    if '@' in username:\n        parts = username.split('@')\n        email = '@'.join(parts[1:])\n    else:\n        # If no @ symbol, assume username is the email\n        email = username"
  }
]

# Apply the patch
modified_content = apply_patch_to_content(file_content, patch_operations)

# Generate the diff
diff = generate_diff(file_content, modified_content, "utils/auth.py")
print(diff)
```<end_code>
Observation:
--- a/utils/auth.py
+++ b/utils/auth.py
@@ -10,8 +10,11 @@
     Returns:
         User object if authentication successful, None otherwise
     """
     # Extract email from username (assuming username is email)
-    parts = username.split('@')
-    email = '@'.join(parts[1:])
+    if '@' in username:
+        parts = username.split('@')
+        email = '@'.join(parts[1:])
+    else:
+        # If no @ symbol, assume username is the email
+        email = username
     
     if not email:
         return None

Thought: The diff looks good. I've modified the code to check if '@' is in the username first. If it is, it proceeds with the original logic of extracting the email part. If not, it just assumes the entire username is the email. This will prevent the IndexError when a username doesn't contain an '@' symbol.
Code:
```py
final_answer({
  "file_path": "utils/auth.py",
  "modified_content": modified_content,
  "diff": diff,
  "description": "Fixed the authentication function to handle usernames without an '@' symbol. The original code assumed all usernames would have an '@' and tried to extract the email part by splitting on '@', which causes an IndexError when no '@' is present. The fix adds a check to see if '@' is in the username before trying to split it, and if not, it just uses the entire username as the email."
})
```<end_code>
---

You must carefully analyze the issue and file to understand the root cause of the problem. Consider edge cases, error handling, and test scenarios when designing your patch. Make sure your patch is minimal and focused on fixing the specific issue.

On top of performing computations in the Python code snippets that you create, you only have access to these tools:
- get_folders: Get all unique folder paths in the repository.
    Takes inputs: {}
    Returns an output of type: array
- get_files_in_folder: Get all files in a specific folder.
    Takes inputs: {'folder_path': {'type': 'string', 'description': 'The relative folder path to query.'}}
    Returns an output of type: array
- get_files_by_extension: Get all files with a specific extension.
    Takes inputs: {'extension': {'type': 'string', 'description': "The file extension to filter by (without the dot, e.g., 'py' not '.py')."}}
    Returns an output of type: array
- get_file_by_path: Get file information for a specific file path.
    Takes inputs: {'file_path': {'type': 'string', 'description': 'The relative path to the file, including folder and filename.'}}
    Returns an output of type: object
- get_file_content: Get the content of a file by path.
    Takes inputs: {'file_path': {'type': 'string', 'description': 'The relative path to the file, including folder and filename.'}}
    Returns an output of type: string
- get_entity_at_line: Find the code entity (function, class, or variable) at a specific line in a file.
    Takes inputs: {'file_path': {'type': 'string', 'description': 'The relative path to the file, including folder and filename.'}, 'line_number': {'type': 'integer', 'description': 'The line number to check.'}}
    Returns an output of type: object
- get_function_by_name: Find functions or methods by name, optionally filtering by class and file.
    Takes inputs: {'function_name': {'type': 'string', 'description': 'The name of the function or method to find.'}, 'class_name': {'type': 'string', 'nullable': True, 'description': 'Optional class name to filter by (for methods).'}, 'file_path': {'type': 'string', 'nullable': True, 'description': 'Optional file path to limit the search to a specific file.'}}
    Returns an output of type: array
- get_class_by_name: Find classes by name, optionally filtering by file.
    Takes inputs: {'class_name': {'type': 'string', 'description': 'The name of the class to find.'}, 'file_path': {'type': 'string', 'nullable': True, 'description': 'Optional file path to limit the search to a specific file.'}}
    Returns an output of type: array
- get_class_methods: Get all methods defined in a specific class.
    Takes inputs: {'class_name': {'type': 'string', 'description': 'The name of the class to find methods for.'}, 'file_path': {'type': 'string', 'nullable': True, 'description': 'Optional file path to limit the search to a specific file.'}}
    Returns an output of type: array
- get_file_structure: Get the complete structure of a Python file including functions, classes, and variables.
    Takes inputs: {'file_path': {'type': 'string', 'description': 'The relative path to the file, including folder and filename.'}}
    Returns an output of type: object
- get_code_segment: Get a segment of code from a file between the specified line numbers.
    Takes inputs: {'file_path': {'type': 'string', 'description': 'The relative path to the file, including folder and filename.'}, 'start_line': {'type': 'integer', 'description': 'The starting line number (1-based).'}, 'end_line': {'type': 'integer', 'description': 'The ending line number (1-based, inclusive).'}}
    Returns an output of type: string
- generate_diff: Generate a unified diff between original and modified content.
    Takes inputs: {'original_content': {'type': 'string', 'description': 'Original file content'}, 'modified_content': {'type': 'string', 'description': 'Modified file content'}, 'file_path': {'type': 'string', 'description': 'Path to the file'}}
    Returns an output of type: string
- extract_method_context: Extract a method and its surrounding context from a file.
    Takes inputs: {'file_content': {'type': 'string', 'description': 'Content of the file'}, 'method_name': {'type': 'string', 'description': 'Name of the method to find'}}
    Returns an output of type: object
- identify_issue_patterns: Identify patterns in the file that may be causing the issue.
    Takes inputs: {'file_content': {'type': 'string', 'description': 'Content of the file'}, 'issue_description': {'type': 'string', 'description': 'Description of the GitHub issue'}}
    Returns an output of type: array
- apply_patch_to_content: Apply a series of patch operations to the file content.
    Takes inputs: {'original_content': {'type': 'string', 'description': 'Original file content'}, 'patch_operations': {'type': 'array', 'items': {'type': 'object', 'additionalProperties': {'type': 'any'}}, 'description': 'List of operations to apply, each being a dict with: - operation: "replace", "insert", or "delete" - start_line: Starting line number (1-indexed) - end_line: Ending line number for replace/delete (1-indexed) - content: New content for replace/insert operations'}}
    Returns an output of type: string
- final_answer: Provides a final answer to the given problem.
    Takes inputs: {'answer': {'type': 'any', 'description': 'The final answer to the problem'}}
    Returns an output of type: any

Here are the rules you should always follow to solve your task:
1. Always provide a 'Thought:' sequence, and a 'Code:\n```py' sequence ending with '```<end_code>' sequence, else you will fail.
2. Use only variables that you have defined!
3. Always use the right arguments for the tools. DO NOT pass the arguments as a dict as in 'answer = wiki({'query': "What is the place where James Bond lives?"})', but use the arguments directly as in 'answer = wiki(query="What is the place where James Bond lives?")'.
4. Take care to not chain too many sequential tool calls in the same code block, especially when the output format is unpredictable. For instance, a call to search has an unpredictable return format, so do not have another tool call that depends on its output in the same block: rather output results with print() to use them in the next block.
5. Call a tool only when needed, and never re-do a tool call that you previously did with the exact same parameters.
6. Don't name any new variable with the same name as a tool: for instance don't name a variable 'final_answer'.
7. Never create any notional variables in our code, as having these in your logs will derail you from the true variables.
8. You can use imports in your code, but only from the following list of modules: ['stat', 'math', 'statistics', 'time', 'queue', 'collections', 'datetime', 'unicodedata', 'random', 're', 'itertools']
9. The state persists between code executions: so if in one step you've created variables or imported modules, these will all persist.
10. Don't give up! You're in charge of solving the task, not providing directions to solve it.

Now Begin! Analyze the issue and file, develop a patch, and generate a diff to fix the problem.

In [6]:
patch_agent.run(task)

╭───────────────────────────────────────────── New run - patch_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ You are tasked with understanding an issue in the following github repository pylint-dev/astroid.               │
│ You will be given a description of the issue and tools to search the codebase.                                  │
│ Your job is to determine, what file or files are causing the issue.                                             │
│ The tools provided will allow you to search the codebase and find the files that are causing the issue.         │
│ Determine what is the function or class where this issue is happening                                           │
│ ### Issue Description                                                                                           │
│ TypeError: unsupported format string passed to NoneType.__format__                                              │
│ Regression in #2459                                                                                             │
│                                                                                                                 │
│ ### Steps to reproduce                                                                                          │
│ a.py:                                                                                                           │
│ ```py                                                                                                           │
│ class A:                                                                                                        │
│     def __init__(self):                                                                                         │
│         self._magnitude = None                                                                                  │
│                                                                                                                 │
│     def name(self) -> str | None:                                                                               │
│         if self._magnitude:                                                                                     │
│             return f"M {self._magnitude:.1f}"                                                                   │
│ ```                                                                                                             │
│ ```                                                                                                             │
│ pylint a.py                                                                                                     │
│ ```                                                                                                             │
│ ### Current behavior                                                                                            │
│ ```                                                                                                             │
│   File "/Users/jwalls/release/lib/python3.12/site-packages/astroid/nodes/node_classes.py", line 4778, in        │
│ _infer_from_values                                                                                              │
│     yield from nodes[0]._infer(context, **kwargs)                                                               │
│   File "/Users/jwalls/release/lib/python3.12/site-packages/astroid/nodes/node_classes.py", line 4695, in _infer │
│     formatted = format(value.value, format_spec.value)                                                          │
│                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                          │
│ TypeError: unsupported format string passed to NoneType.__format__                                              │
│ ```                                                   

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  file_content = get_file_content(file_path="astroid/nodes/node_classes.py")                                       
  print(file_content)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
# Licensed under the LGPL: https://www.gnu.org/licenses/old-licenses/lgpl-2.1.en.html
# For details: https://github.com/pylint-dev/astroid/blob/main/LICENSE
# Copyright (c) https://github.com/pylint-dev/astroid/blob/main/CONTRIBUTORS.txt

"""Module for some node classes. More nodes in scoped_nodes.py"""

from __future__ import annotations

import abc
import ast
import itertools
import operator
import sys
import typing
import warnings
from collections.abc import Callable, Generator, Iterable, Iterator, Mapping
from functools import cached_property
from typing import (
    TYPE_CHECKING,
    Any,
    ClassVar,
    Literal,
    Optional,
    Union,
)

from astroid import decorators, protocols, util
from astroid.bases import Instance, _infer_stmts
from astroid.const import _EMPTY_OBJECT_MARKER, Context
from astroid.context import CallContext, InferenceContext, copy_context
from astroid.exceptions import (
    AstroidBuildingError,
    AstroidError,
    AstroidIndexError,
    AstroidTypeError,
    AstroidValueError,
    AttributeInferenceError,
    InferenceError,
    NameInferenceError,
    NoDefault,
    ParentMissingError,
    _NonDeducibleTypeHierarchy,
)
from astroid.interpreter import dunder_lookup
from astroid.manager import AstroidManager
from astroid.nodes import _base_nodes
from astroid.nodes.const import OP_PRECEDENCE
from astroid.nodes.node_ng import NodeNG
from astroid.typing import (
    ConstFactoryResult,
    InferenceErrorInfo,
    InferenceResult,
    SuccessfulInferenceResult,
)

if sys.version_info >= (3, 11):
    from typing import Self
else:
    from typing_extensions import Self

if TYPE_CHECKING:
    from astroid import nodes
    from astroid.nodes import LocalsDictNodeNG


def _is_const(value) -> bool:
    return isinstance(value, tuple(CONST_CLS))


_NodesT = typing.TypeVar("_NodesT", bound=NodeNG)
_BadOpMessageT = typing.TypeVar("_BadOpMessageT", bound=util.BadOperationMessage)

AssignedStmtsPossibleNode = Union["List", "Tuple", "AssignName", "AssignAttr", None]
AssignedStmtsCall = Callable[
    [
        _NodesT,
        AssignedStmtsPossibleNode,
        Optional[InferenceContext],
        Optional[list[int]],
    ],
    Any,
]
InferBinaryOperation = Callable[
    [_NodesT, Optional[InferenceContext]],
    Generator[Union[InferenceResult, _BadOpMessageT], None, None],
]
InferLHS = Callable[
    [_NodesT, Optional[InferenceContext]],
    Generator[InferenceResult, None, Optional[InferenceErrorInfo]],
]
InferUnaryOp = Callable[[_NodesT, str], ConstFactoryResult]


@decorators.raise_if_nothing_inferred
def unpack_infer(stmt, context: InferenceContext | None = None):
    """recursively generate nodes inferred by the given statement.
    If the inferred value is a list or a tuple, recurse on the elements
    """
    if isinstance(stmt, (List, Tuple)):
        for elt in stmt.elts:
            if elt is util.Uninferable:
                yield elt
                continue
            yield from unpack_infer(elt, context)
        return {"node": stmt, "context": context}
    # if inferred is a final node, return it and stop
    inferred = next(stmt.infer(context), util.Uninferable)
    if inferred is stmt:
        yield inferred
        return {"node": stmt, "context": context}
    # else, infer recursively, except Uninferable object that should be returned as is
    for inferred in stmt.infer(context):
        if isinstance(inferred, util.UninferableBase):
            yield inferred
        else:
            yield from unpack_infer(inferred, context)

    return {"node": stmt, "context": context}


def are_exclusive(stmt1, stmt2, exceptions: list[str] | None = None) -> bool:
    """return true if the two given statements are mutually exclusive

    `exceptions` may be a list of exception names. If specified, discard If
    branches and check one of the statement is in an exception handler catching
    one of the given exceptions.

    algorithm :
     1) index stmt1's parents
     2) climb amon

[Step 0: Duration 29.01 seconds| Input tokens: 4,241 | Output tokens: 178]

...

Execution logs:
[FunctionInfo(id=824, name='_infer', start_line=346, end_line=367, args='self, context', is_method=True, 
class_name='BaseContainer', is_async=False, decorators='decorators.raise_if_nothing_inferred', docstring=None, 
file_id=135, file_path='astroid/nodes/node_classes.py'), FunctionInfo(id=827, name='_infer', start_line=447, 
end_line=457, args='self, context', is_method=True, class_name='AssignName', is_async=False, 
decorators='decorators.raise_if_nothing_inferred, decorators.path_wrapper', docstring='Infer an AssignName: need to
inspect the RHS part of the\nassign node.', file_id=135, file_path='astroid/nodes/node_classes.py'), 
FunctionInfo(id=832, name='_infer', start_line=575, end_line=601, args='self, context', is_method=True, 
class_name='Name', is_async=False, decorators='decorators.raise_if_nothing_inferred, decorators.path_wrapper', 
docstring='Infer a Name: use name lookup rules\n\nSame implementation as AssignName._infer_lhs.', file_id=135, 
file_path='astroid/nodes/node_classes.py'), FunctionInfo(id=844, name='_infer', start_line=1029, end_line=1037, 
args='self, context', is_method=True, class_name='Arguments', is_async=False, 
decorators='decorators.raise_if_nothing_inferred', docstring=None, file_id=135, 
file_path='astroid/nodes/node_classes.py'), FunctionInfo(id=851, name='_infer', start_line=1172, end_line=1182, 
args='self, context', is_method=True, class_name='AssignAttr', is_async=False, 
decorators='decorators.raise_if_nothing_inferred, decorators.path_wrapper', docstring='Infer an AssignAttr: need to
inspect the RHS part of the\nassign node.', file_id=135, file_path='astroid/nodes/node_classes.py'), 
FunctionInfo(id=869, name='_infer', start_line=1448, end_line=1453, args='self, context', is_method=True, 
class_name='AugAssign', is_async=False, decorators='decorators.raise_if_nothing_inferred, decorators.path_wrapper',
docstring=None, file_id=135, file_path='astroid/nodes/node_classes.py'), FunctionInfo(id=877, name='_infer', 
start_line=1563, end_line=1568, args='self, context', is_method=True, class_name='BinOp', is_async=False, 
decorators='decorators.yes_if_nothing_inferred, decorators.path_wrapper', docstring=None, file_id=135, 
file_path='astroid/nodes/node_classes.py'), FunctionInfo(id=882, name='_infer', start_line=1640, end_line=1690, 
args='self, context', is_method=True, class_name='BoolOp', is_async=False, 
decorators='decorators.raise_if_nothing_inferred, decorators.path_wrapper', docstring='Infer a boolean operation 
(and / or / not).\n\nThe function will calculate the boolean operation\nfor all pairs generated through inference 
for each component\nnode.', file_id=135, file_path='astroid/nodes/node_classes.py'), FunctionInfo(id=887, 
name='_infer', start_line=1751, end_line=1774, args='self, context', is_method=True, class_name='Call', 
is_async=False, decorators='decorators.raise_if_nothing_inferred, decorators.path_wrapper', docstring='Infer a Call
node by trying to guess what the function returns.', file_id=135, file_path='astroid/nodes/node_classes.py'), 
FunctionInfo(id=894, name='_infer', start_line=1912, end_line=1936, args='self, context', is_method=True, 
class_name='Compare', is_async=False, decorators='', docstring='Chained comparison inference logic.', file_id=135, 
file_path='astroid/nodes/node_classes.py'), FunctionInfo(id=906, name='_infer', start_line=2169, end_line=2172, 
args='self, context', is_method=True, class_name='Const', is_async=False, decorators='', docstring=None, 
file_id=135, file_path='astroid/nodes/node_classes.py'), FunctionInfo(id=924, name='_infer', start_line=2434, 
end_line=2449, args='self, context', is_method=True, class_name='Dict', is_async=False, decorators='', 
docstring=None, file_id=135, file_path='astroid/nodes/node_classes.py'), FunctionInfo(id=933, name='_infer', 
start_line=2562, end_line=2573, args='self, context', is_method=True, class_name='EmptyNode', is_async=False, 
decorators='decorators.raise_if_nothing_inferred, de

[Step 5: Duration 1.67 seconds| Input tokens: 100,490 | Output tokens: 1,101]

Reached max steps.

[Step 6: Duration 1.67 seconds| Input tokens: 120,578 | Output tokens: 1,227]

'Okay, I have a list of `_infer` methods. The one I\'m interested in is in `FormattedValue`, which starts on line 4676 and ends on line 4702. The relevant line in the traceback is line 4695. Now I can extract the code segment.\nCode:\n```py\ncode_segment = get_code_segment(file_path="astroid/nodes/node_classes.py", start_line=4676, end_line=4702)\nprint(code_segment)\n```'

In [ ]:
print(issue_data.loc[0]['patch'])

In [4]:
if os.path.exists('repository.db'):
    os.remove('repository.db')


In [ ]:
repo_path

In [ ]:
parse_repository(repo_path)
analyze_python_files()

In [7]:

SYSTEM_PROMPT = """You are an expert assistant who can solve any task using code blobs.
You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code. To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.
At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use. Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the final_answer tool."""

In [ ]:

model = create_model()

analysis_agent = create_analysis_agent(model=model)

analysis_agent.visualize()

In [9]:
# llm_model_pth = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
agent = create_code_agent(model=model, analysis_agent=analysis_agent)

In [ ]:
agent.visualize()

In [ ]:
Markdown(agent.system_prompt)

In [ ]:
row

In [13]:
task = f"""You are tasked with understanding an issue in the following github repository {row['repo']}.
You will be given a description of the issue and tools to search the codebase.
Your job is to determine, what file or files are causing the issue.
The tools provided will allow you to search the codebase and find the files that are causing the issue.
Determine what is the function or class where this issue is happening
### Issue Description
{row['problem_statement']}"""

In [ ]:
agent.run(task)